Today we will be exploring the use of high-throughput sequencing data as a biochemical readout from the genome. 

A large resource for these types of data was generated by the ENCODE consortium.


In today's lab project, we will be walking through a specific experimental hypothesis to demonstrate useful tools for analysis of high-throughput sequecning data. We will be skipping the general steps of alignment of sequencing data to a genome which are identical to the previous lab. If you recall, one of the primary datatypes from the previous lab was the BAM file which represented the binary format of the sequencing data aligned to the genome. The sequences in the previous lab represented whole-genome sequencing data and today's data are from enriched genomic regions, but the steps to generate the original BAM alignments are identical.


Out goal today is to identify enrichment of specific histone modifications at features throughout the genome.

If you recall from the lecture, specific histone modifications are indiciative of specific functions throughout the genome. 

The general steps of this process will be as follows:

1. Retrieve aligned sequence data from a specific histone modification from ENCODE.
2. Generate a fold-enrichment genome 'track' for the histone of interest.
3. Retrieve open chromatin information from ENCODE.
4. Generate the enrichment of the histone modification at open chromatin regions.
5. Visualize these regions.

A method for interacting with the ENCODE data is directly through the ENCODE DCC web interface. For simplicity, we will download data directly from the ENCODE DCC using the web interface. I will also provide an API script for future downloads if you want to write a script for this. I will demonstrate this system and we will download some sequence data there: https://www.encodeproject.org/

In [ ]:
#! wget https://www.encodeproject.org/files/ENCFF000BXW/@@download/ENCFF000BXW.bam
! cp /class/projects/BINF527_ENCODE_F15/ENCFF000BXW.bam ./

We will also get a control data set for the H3K4me3 data for generating our signal tracks.

In [ ]:
#! wget https://www.encodeproject.org/files/ENCFF000BVZ/@@download/ENCFF000BVZ.bam
! cp /class/projects/BINF527_ENCODE_F15/ENCFF000BVZ.bam ./   

You can use the QC tool Ryan introduced last week. For something different, we can check the number of mapped reads directly. ENCODE guidelines require at least 20 million mapped reads for 'peaky' data and 40 million mapped reads for 'broad' data.

We can also download pre-processed data types. Today we will download DNase-seq data from K562 that has had peaks called for us already.

In [ ]:
! wget -q https://www.encodeproject.org/files/ENCFF001UWQ/@@download/ENCFF001UWQ.bed.gz

We will first be collecting data from the ENCODE API resource. The REST API, described at https://www.encodeproject.org/help/rest-api/, allows for automated queries against the database in the form of HTTP GET requests. Query Results are returned in JSON format, a tree-like format where key:value pairs are stored in a nested hierarchy, which can be parsed using readily available modules for PERL, Python and many other popular programming languages. The actual queries are packaged as parameters to the URL, which will look very familiar to those who have spent time using the ENCODE faceted search. In fact, the minimal URL to retrieve results through the API differs only in the inclusion of the “&format=json” parameter. This means that, by adding or removing this tag, search URLs from the faceted search can be plugged directly into the API and vice-versa, giving a convenient way to (p)review results.



We will start by using git to retrieve a simple script that handles the ENCODE API for us:

In [ ]:
! git clone https://github.com/adadiehl/ENCODE-API-Apps.git

In [ ]:
! ls ENCODE-API-Apps

In [ ]:
! perl ENCODE-API-Apps/search_encode.pl 

In [ ]:
! perl ENCODE-API-Apps/search_encode.pl "K562" "&assay_term_name=ChIA-PET&files.file_type=bigBed bed12"

In [ ]:
! ls

Once we are happy with the data set given our extensive quality control checks, we will start to process these data. The BAM files represent aligned sequence reads in the genome. In order to look at their specific enrichment at our genomic feature we can convert the reads to a signal density. One of the tools to generate this denisty is the ChIP-seq peak caller MACS. We will use MACS to create a fold enrichment model of signal density with respect to the input data.

In [ ]:
! macs2 callpeak -t ENCFF000BXW.bam -c ENCFF000BVZ.bam -B --SPMR -g mm -n H3K4me3
! macs2 bdgcmp -t H3K4me3_treat_pileup.bdg -c H3K4me3_control_lambda.bdg -o H3K4me3_FE.bedgraph -m FE

In [ ]:
! ls -lah

The fold enrichment bedgraph file that we generated contains a summary of the signal strength of the H3K4me3 mark across the genome.

In [ ]:
! head H3K4me3_FE.bedgraph

Now we can look at just the enrichment of the histone mark in DNase hypersensitive sites. To do this we can use the powerful suit of tools called bedtools. See: http://bedtools.readthedocs.org/en/latest/index.html#

The first tool we will use is intersect. Intersect takes the merge across genomic intervals. This means that anything in feature 'A' that overlaps feature 'B' will be reported.

In [ ]:
! bedtools intersect -a H3K4me3_FE.bedgraph -b ENCFF001UWQ.bed.gz -wa -u | head

In [ ]:
! bedtools intersect -a H3K4me3_FE.bedgraph -b ENCFF001UWQ.bed.gz -wa -u > H3K4me3_in_DNase.bedgraph

Unfortunately, MACS signal output is in a format called 'bedgraph' which isn't the appropriate input for our future data sets. We will need to convert this into a 'wiggle' format.

In [ ]:
! fetchChromSizes hg19 > hg19.chrom.sizes

In [ ]:
! cat hg19.chrom.sizes

In [ ]:
! bedtools sort -i H3K4me3_in_DNase.bedgraph > H3K4me3_in_DNase.sorted.bedgraph

In [ ]:
! bedGraphToBigWig H3K4me3_in_DNase.sorted.bedgraph hg19.chrom.sizes H3K4me3_in_DNase.bw

In [ ]:
! ls -la

We can now visualize this in the UCSC genome browser: http://genome.ucsc.edu/index.html

We need to externally host the file (eg. in your box or dropbox account) to actually vizualize it. In the meantime, we can use this link: https://dl.dropboxusercontent.com/u/8566399/H3K4me3_in_DNase.bw

BEDTools is very powerful. Another question that we might have is to look at the average signal across certain features. We can do this by using the orginal BAM file from H3K4me3 and the first 10 DNase peaks. 

In [ ]:
! gunzip -c ENCFF001UWQ.bed.gz | head -n 10 | bedtools coverage -a stdin -b ENCFF000BXW.bam -mean